In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-09-08 12:58:58--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.8’

book-crossings.zip. 100%[===================>]  24.88M  54.1MB/s    in 0.5s    

2025-09-08 12:58:59 (54.1 MB/s) - ‘book-crossings.zip.8’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
# add your code here - consider creating a new cell for each section of code
# --- Filter simultaneously ---
# --- Filter simultaneously ---
# --- Filter out inactive users and unpopular books ---
ratings_per_user = df_ratings.groupby("user")["rating"].count()
df_ratings = df_ratings[df_ratings["user"].isin(ratings_per_user[ratings_per_user >= 200].index)]

ratings_per_book = df_ratings.groupby("isbn")["rating"].count()
df_ratings = df_ratings[df_ratings["isbn"].isin(ratings_per_book[ratings_per_book >= 100].index)]


df_ratings

,user,isbn,rating
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1495,277427,0142001740,0.0
1513,277427,0312966091,0.0
...,...,...,...
1146823,275970,0440221471,0.0
1146824,275970,0440222656,0.0
1146825,275970,0440226430,0.0
1146852,275970,044651652X,6.0


In [ ]:
df_ratings.shape

(13793, 3)

In [ ]:
# --- Create book-user matrix ---
book_user_matrix = df_ratings.pivot(index="isbn", columns="user", values="rating").fillna(0)

# --- Sparse matrix for efficiency ---
book_matrix = csr_matrix(book_user_matrix.values)


In [ ]:
book_user_matrix

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
0060502258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060928336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060930535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060934417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060976845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
080410526X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0804106304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
080410753X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(book_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
isbn_to_title = dict(zip(df_books["isbn"], df_books["title"]))

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book_title, n_recommendations=5):
    # Get ISBN(s) for the given title
    book_isbns = df_books[df_books["title"] == book_title]["isbn"].values

    if len(book_isbns) == 0:
        return f"❌ Book '{book_title}' not found in dataset."

    # Pick the first ISBN that survived filtering
    book_isbn = None
    for isbn in book_isbns:
        if isbn in book_user_matrix.index:
            book_isbn = isbn
            break

    if book_isbn is None:
        return f"⚠️ '{book_title}' not found in filtered dataset."

    # Find index of this ISBN in the matrix
    book_idx = book_user_matrix.index.get_loc(book_isbn)

    # Query nearest neighbors
    distances, indices = model.kneighbors(
        book_user_matrix.iloc[book_idx, :].values.reshape(1, -1),
        n_neighbors=n_recommendations + 1
    )

    # Map ISBNs back to titles
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        rec_isbn = book_user_matrix.index[indices.flatten()[i]]
        rec_title = df_books.loc[df_books["isbn"] == rec_isbn, "title"].values[0]
        recommended_books.append([
            rec_title,
            float(distances.flatten()[i])  # ensure plain float
        ])

    return [book_title, recommended_books]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ['The Lovely Bones: A Novel', "The Pilot's Wife : A Novel", 'The Joy Luck Club', 'I Know This Much Is True']
  recommended_books_dist = [0.77, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.06:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864234924316], ["The Pilot's Wife : A Novel", 0.8192678689956665], ['The Joy Luck Club', 0.8198604583740234], ['The Notebook', 0.8236682415008545], ['Bel Canto: A Novel', 0.8247874975204468]]]
You passed the challenge! 🎉🎉🎉🎉🎉


In [ ]:
s = input("Enter a book title: ")
print(get_recommends(s))

Enter a book title: The Lovely Bones: A Novel
['The Lovely Bones: A Novel', [["Where the Heart Is (Oprah's Book Club (Paperback))", 0.7234864234924316], ["The Book of Ruth (Oprah's Book Club (Paperback))", 0.7633429765701294], ['Good in Bed', 0.76594477891922], ["The Pilot's Wife : A Novel", 0.7673946619033813], ['Lucky : A Memoir', 0.7762122750282288]]]
